## Motivation for recurrent layers

![](../../images/35.png)

![](../../images/36.png)

> Question 1

How many weights are there in the first layer of the MLP? Do not forget about the bias vector!

hidden neurons: 100

window width: 100

word embeddings size: 100


- 10 100

- 10 000

- __1 000 100__

__There are 100*100 inputs to the first layer of the MLP: 100 elements for each word in the window. Therefore the weight matrix contains 1M parameters. Additionally, the bias vector contains 100 parameters.__

- 1 000 000


![](../../images/37.png)

![](../../images/38.png)


> Question 2

How many weights are there in the first layer of the MLP? Do not forget about the bias vector.

hidden neurons: 100

word embeddings size: 100

we transfer hidden neurons of the first layer between time steps

- __20 100__

__There are 200 inputs to the first layer of the MLP: 100 from the current input and 100 from the previous hidden state. Therefore the weight matrix contains 200*100 parameters. Additionally, the bias vector contains 100 parameters.__

- 10 100

- 10 200

- 20 200



Takeaways:

![](../../images/39.png)


## Simple RNN and Backpropagation

![](../../images/40.png)

![](../../images/41.png)

![](../../images/42.png)


> Note:

All weights are shared across time steps


![](../../images/43.png)

![](../../images/44.png)


![](../../images/45.png)


![](../../images/46.png)

![](../../images/47.png)


![](../../images/48.png)






## The training of RNN

### To train an RNN we need to backprop through layers and time


![](../../images/49.png)


__The more steps between the time moments k and t, the more elements are in this product__

> Gradient Vanishing and Exploding !!

- Values of these Jacobian matrices have particularly severe impact on the contribution from faraway steps

![](../../images/50.png)

![](../../images/51.png)

![](../../images/52.png)




## Dealing with vanishing and exploding gradients


### Gradient Exploding

![](../../images/53.png)

![](../../images/54.png)

![](../../images/55.png)

### Gradient Vanishing

![](../../images/56.png)

![](../../images/57.png)


> How can we deal with gradient vanishing and exploding?

- LSTM, GRU
- RELU activation function
- Initialization of the recurrent weight matrix
- Skip connections



#### Property of orthogonal matrix

![](../../images/58.png)

So no matter how many times we perform repeated matrix multiplication, the resulting matrix does not explode or vanish.


- Initialize W with an orthogonal matrix
- Use orthogonal W through the whole training

----

In this way the second part of the Jacobian doesn't cause the vanishing gradient problem, at least on the first iterations of the training.



__In this case, the network has a chance to find long range dependencies in the data.__

----

There are some approaches that utilize the properties of orthogonal matrices, not just for a proper initialization, but also for the parameterization of the weights for the whole training process.

----
__Takeaways:__

- Exploding gradients are easy to detect but it is not clear how to detect vanishing gradients
- Exploding gradients: 
    - gradient clipping 
    - truncated BPTT
- Vanishing gradients:
    - ReLU nonlinearity
    - orthogonal initialization of the recurrent weights
    - skip connection






## Modern RNNs: LSTM and GRU

![](../../images/59.png)

When we do a back-prop on such layer, __gradient needs to go through non-linearity__ and __for multiplication by weight matrix W__, both can cause `gradient vanishing`


### So the main idea is to create a short way for the gradients result any non-linearities or multiplications.


- All sorts of LSTM propose to do it by adding a new separately way through recurrent layer --> __Interal memory C__ (__which other layers on the network do not have access to__)


- At each step of LSTM, we compute not only the vector of hidden units H but also a vector of memory c on the same dimension

As the result, we have two ways through such layer:

- one between $H_{t-1}$ and $H_t$
- second between the memory cells $C_{t-1}$ and $C_t$

![](../../images/60.png)



![](../../images/61.png)


![](../../images/62.png)

![](../../images/63.png)




## 